<a href="https://colab.research.google.com/github/Tarunno360/Natural-Language-Processing/blob/main/Bangla%20Grammar%20Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Not working part of the research

In [ ]:
#downloadable content


#task1
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
#task2
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
#task3
!pip install spacy
import spacy
!python -m spacy download xx_ent_wiki_sm # Download the model

#task4
!pip install bert-score
from bert_score import score

#sanza
!pip install stanza

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 24.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#bangla bert is not working as per our goal

model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForPreTraining.from_pretrained(model_name)

def calculate_perplexity(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
    input_ids = inputs["input_ids"]

    mask_token_index = tokenizer.mask_token_id
    total_loss = 0
    for i in range(1, len(input_ids[0]) - 1):
      masked_input_ids = input_ids.clone()
      masked_input_ids[0, i] = mask_token_index

      with torch.no_grad():
        outputs = model(masked_input_ids)
        discriminator_logits = outputs.logits

        predicted_token_logits = discriminator_logits[0,i]
        true_token_label = torch.tensor([1]).to(predicted_token_logits.device)

        loss = torch.nn.functional.binary_cross_entropy_with_logits(predicted_token_logits.unsqueeze(0), true_token_label.float())

        total_loss += loss

    avg_loss = total_loss / (len(input_ids[0]) - 2)
    return torch.exp(avg_loss).item()

sentence = "আপনি কি বাংলা ভাষা বোঝেন?"
print("Perplexity:", calculate_perplexity(sentence))


Perplexity: 67611.1171875



Perplexity Score (Mathematical) Use a Bangla language model (gpt2-benglai
) Calculate perplexity: Lower scores indicate better grammar. Implementation

N-gram Matching (Statistical)
Compare MCQ structures with a reference corpus using n-gram probabilities.
Use BLEU or ROUGE scores.


In [ ]:
#perfectly worked
reference = ["বাংলা ভাষা সহজ ও সুন্দর।"]
candidate = ["বাংলা ভাষা সহজ এবং সুন্দর।"]

# Tokenize the sentences
reference_tokens = [nltk.word_tokenize(sent) for sent in reference]
candidate_tokens = nltk.word_tokenize(candidate[0])


# Calculate BLEU score with only unigrams
smoothing_function = SmoothingFunction().method1
bleu_score_unigram = sentence_bleu(reference_tokens, candidate_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothing_function)
print("BLEU Score (Unigram):", bleu_score_unigram)


BLEU Score (Unigram): 0.8


3. Syntax Tree Matching (Linguistic)
Use a Bangla dependency parser to check grammatical correctness.
Compare MCQs’ syntax trees with correct structures.
Example using SpaCy:

In [ ]:

#perfectly worked
# Load the SpaCy model
nlp = spacy.load("xx_ent_wiki_sm")

def check_structure(sentence):
    """
    Checks if a sentence's dependency structure generally follows a subject-object-verb (SOV)
    structure with appropriate dependencies
    """
    doc = nlp(sentence)




    subject_index = -1
    object_index = -1
    verb_index = -1



    actual_dependencies = [token.text + " " + token.dep_ for token in doc]

    for i, token in enumerate(doc):
      if token.dep_ == 'nsubj':
        subject_index = i
      elif token.dep_ == 'obj':
        object_index = i
      elif token.dep_ == 'ROOT':
        verb_index = i

    if subject_index != -1 and object_index != -1 and verb_index != -1 and subject_index < object_index < verb_index :
      # We assume if all three are present and in SOV order, then SOV structure has occurred
        return True, actual_dependencies
    elif subject_index != -1 and verb_index != -1 and subject_index < verb_index and object_index == -1:
      return True, actual_dependencies
    else:
        return False, actual_dependencies


# Example sentences
reference_sentences = ["আমি ভাত খাই।"] #Subject Object Verb
candidate_sentences = ["আপনি কি বাংলা ভাষা বোঝেন?", "আমি বাংলায় কথা বলি।"] #subject verb object

for i, ref in enumerate(reference_sentences):
  is_correct, ref_dependencies = check_structure(ref)
  print(f"Reference {i+1}: '{ref}' - SOV Structure: {is_correct}, Dependencies: {ref_dependencies}")

for i, cand in enumerate(candidate_sentences):
  is_correct, cand_dependencies = check_structure(cand)
  print(f"Candidate {i+1}: '{cand}' - SOV Structure: {is_correct}, Dependencies: {cand_dependencies}")

Reference 1: 'আমি ভাত খাই।' - SOV Structure: False, Dependencies: ['আমি ', 'ভাত ', 'খাই ', '। ']
Candidate 1: 'আপনি কি বাংলা ভাষা বোঝেন?' - SOV Structure: False, Dependencies: ['আপনি ', 'কি ', 'বাংলা ', 'ভাষা ', 'বোঝেন ', '? ']
Candidate 2: 'আমি বাংলায় কথা বলি।' - SOV Structure: False, Dependencies: ['আমি ', 'বাংলায় ', 'কথা ', 'বলি ', '। ']


4. BERTScore (Deep Learning-Based)
Measures semantic correctness (better than BLEU/ROUGE).
Compares MCQs with correct reference MCQs.

In [ ]:
#Perfectly worked

references = ["আপনি কি বাংলা ভাষা বোঝেন?"]
candidates = ["আপনি কি বাংলা ভাষা বুঝতে পারেন?"]

P, R, F1 = score(candidates, references, lang="bn", rescale_with_baseline=False) # Changed this line
print(f"BERTScore (No Rescaling): Precision={P.mean():.4f}, Recall={R.mean():.4f}, F1={F1.mean():.4f}")

BERTScore (No Rescaling): Precision=0.9283, Recall=0.9584, F1=0.9431


Parser approch


In [ ]:
import stanza

def setup_stanza():
    """Download and initialize Stanza for Bangla language processing."""
    stanza.download("bn")  # Download Bangla model if not already available
    return stanza.Pipeline(lang="bn", processors="tokenize,pos,depparse")

def check_grammar(mcq_text, nlp):
    """Check for possible grammar issues using Stanza dependency parsing."""
    doc = nlp(mcq_text)
    errors = []

    for sentence in doc.sentences:
        for word in sentence.words:
            # Example: Detect unusual root dependencies
            if word.deprel == "root" and word.upos not in ["VERB", "NOUN"]:
                errors.append(f"Possible grammar issue at '{word.text}' ({word.upos})")

            # Detect missing subject (nsubj) in MCQ structure
            if word.deprel == "nsubj" and word.upos != "NOUN":
                errors.append(f"Subject issue detected at '{word.text}' ({word.upos})")

    return errors

if __name__ == "__main__":
    # Initialize Stanza NLP model
    nlp = setup_stanza()

    # Example Bangla MCQ sentence
    bangla_mcq = "বাংলা ভাষা কি একটি আন্তর্জাতিক ভাষা?"  # Example MCQ
    errors = check_grammar(bangla_mcq, nlp)

    # Output detected grammar issues
    if errors:
        print("Grammar Issues Found:")
        for error in errors:
            print(error)
    else:
        print("No grammar issues detected.")


INFO:stanza:Downloading default packages for language: bn (Bengali) ...


KeyError: 'default_md5'

Result: Stanza now does not support bangla language

New approch

In [ ]:
import nltk
from nltk import CFG

def evaluate_bangla_sentence(sentence):
    # Define Bangla grammar using CFG
    bangla_grammar = CFG.fromstring("""
        S -> NP VP
        NP -> Pronoun | Noun
        VP -> Verb | Verb NP
        Pronoun -> 'আমি' | 'তুমি' | 'সে'
        Noun -> 'ভাত' | 'জল' | 'বই'
        Verb -> 'খাই' | 'পড়ি' | 'লিখি'
    """)

    # Tokenize the input sentence
    words = sentence.split()

    # Check if the sentence follows the grammar
    parser = nltk.ChartParser(bangla_grammar)

    for tree in parser.parse_all(words):
        print("Valid sentence structure detected!\n")
        print(tree)
        return 100  # Return 100% accuracy for correct grammar

    # If no valid parse tree is found, return lower accuracy
    print("Incorrect sentence structure!")
    return 50  # Example score for incorrect structure

# Example Sentences
test_sentence_1 = "আমি ভাত খাই"
test_sentence_2 = "ভাত আমি খাই"  # Incorrect word order

accuracy_1 = evaluate_bangla_sentence(test_sentence_1)
accuracy_2 = evaluate_bangla_sentence(test_sentence_2)

Incorrect sentence structure!
Incorrect sentence structure!


# **Working part of the Research**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
#Perfectly Worked
model_name = "flax-community/gpt2-bengali" # This model is causal
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def calculate_perplexity(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        loss = model(**inputs, labels=inputs["input_ids"]).loss
    return torch.exp(loss).item()

sentence = "ওর মাথায় বুধা চাঁটি বাবার মেরে গিয়ে সামনে দাঁড়ায়।"
print("Perplexity:", calculate_perplexity(sentence))

Perplexity: 9.386212348937988


# UDpipe

In [ ]:
!pip install ufal.udpipe
!wget https://lindat.mff.cuni.cz/services/udpipe/models/udpipe-2.5-191206/bn/bn-ud-2.5-191206.udpipe


--2025-04-07 21:08:48--  https://lindat.mff.cuni.cz/services/udpipe/models/udpipe-2.5-191206/bn/bn-ud-2.5-191206.udpipe
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-04-07 21:08:49 ERROR 404: Not Found.



In [ ]:
import ufal.udpipe

# Load the pre-trained model
model = ufal.udpipe.Model.load("bn-ud-2.5-191206.udpipe")
if not model:
    raise Exception("Cannot load model")

# Create a pipeline to process the text
pipeline = ufal.udpipe.Pipeline(model, "tokenize", ufal.udpipe.Pipeline.DEFAULT, ufal.udpipe.Pipeline.DEFAULT, "conllu")

# Example Bengali text
text = "আমি স্কুলে যাই।"

# Process the text
processed_text = pipeline.process(text)
print(processed_text)


Exception: Cannot load model